# 技术人员路由和调度问题

## 目标和前提条件

尝试这个建模示例,了解数学优化如何帮助电信公司自动化并改进其技术人员分配、调度和路由决策,以确保最高水平的客户满意度。

这是一个中级建模示例,我们假设您了解Python并熟悉Gurobi Python API。此外,您对构建数学优化模型有一定了解。要充分理解本笔记本的内容,您应该熟悉面向对象编程。

**下载代码库** <br /> 
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

## 动机
技术人员路由和调度(TRS)是电信公司面临的一个常见问题,他们必须能够为大量客户提供服务。在资源有限的情况下,这些公司必须努力提供及时、经济和可靠的服务,以最大限度地提高客户满意度。TRS问题涉及多技能技术人员的分配、调度和路由,以服务于具有独特优先级、服务时间窗口、处理时间、技能要求和地理位置的客户。
在当前实践中,这些决策往往由操作人员通过启发式方法做出,有时是手动完成的 - 这种方法
不仅耗时,
而且可能显著偏离最优解。
使用数学优化来解决
TRS问题将
使
电信运营商和调度员能够自动化并改进其分配、调度和路由决策,同时实现高客户满意度。

## 问题描述
一家电信公司运营多个服务中心为其客户提供服务。
每个服务中心都有
其技术人员从服务中心派遣到他们被分配的工作地点,并在所有工作完成后返回中心。
技术人员具有多种技能和可用的工作容量,在调度期间不能超过。
服务订单/工作有
已知的处理时间,
客户指定的服务开始时间窗口,
完成服务的截止时间,
及其技能要求。
根据工作的性质(例行维护或紧急情况)和与客户的关系(现有客户、新客户),公司评估工作的重要性并为其分配优先级分数。
一个工作最多分配给一个具备所需
技能的技术人员,
但技术人员在调度期间的可用容量不能超过。
为了解决TRS问题,电信公司必须能够
同时做出三种决策:(i)在所有服务中心分配工作给技术人员;
(ii)每个技术人员的路由,即技术人员访问客户的顺序;
(iii)和工作的调度,即技术人员到达客户位置并完成相应工作的时间。
公司的目标是最小化所有工作的总加权延迟,优先级作为权重。

必须满足以下约束条件:
* 如果使用技术人员,他/她从其所在的服务中心出发,在完成分配的工作后返回同一服务中心。
* 技术人员在调度期间的可用容量不能超过。
* 如果选择了一个工作,它最多分配给一个具备所需技能的技术人员。
* 技术人员必须在客户指定的时间窗口内到达客户位置,并在客户要求的截止时间前完成工作。这是保证客户满意度的重要约束条件。

上述基本TRS是具有时间窗口的多仓库车辆路径问题的变体,在文献中称为MDVRPTW [1]。


## 解决方案方法

数学编程是一种声明性方法,模型构建者通过构建数学优化模型来捕捉复杂决策问题的关键方面。Gurobi优化器使用最先进的数学和计算机科学来解决这些模型。

数学优化模型有五个组成部分,即:

* 集合和索引。
* 参数。
* 决策变量。
* 目标函数。
* 约束条件。

为了简化MIP公式,我们考虑对于每个工作的技能要求,我们可以根据技术人员具备的技能确定有资格执行该工作的技术人员子集。

现在我们为基本的TRS问题提出一个MIP公式。

## 模型公式


### 集合和索引
$j \in J = \{1,2,...,n\}$: 工作的索引和集合。

$k \in K$: 技术人员的索引和集合。

$K(j) \subset K$: 有资格执行工作 $j \in J$ 的技术人员子集

$d \in D = \{n+1, n+2, ...,n+m \}$: 仓库(服务中心)的索引和集合,其中 $m$ 是仓库的数量。

$l, i, j \in L = J \cup D = \{1,2,..., n+m \}$: 访问位置的索引和集合。

### 参数
$\beta_{k,d} \in \{0,1\}$: 如果技术人员 $k \in K$ 必须从仓库 $d \in D$ 出发并返回,则该参数为1;否则为0。

$p_{j} \in \mathbb{R^{+}}$; 工作 $j \in J$ 的处理时间(持续时间)。

$\tau_{i,j} \in \mathbb{R^{+}}$: 从位置 $i \in L$ 到位置 $j \in L$ 的旅行时间(分钟)。

$W_{k} \in \mathbb{N}$: 技术人员 $k \in K$ 在计划期间的工作负荷限制。即技术人员在下一个工作日可用的小时数(分钟)。

$\pi_{j} \in \{1,2,3,4\}$: 工作 $j \in J$ 的优先级权重,数字越大表示优先级越高。

$a_{j} \in \mathbb{R^{+}} $: 在工作 $j \in J$ 位置开始服务的最早时间。

$b_{j} \in \mathbb{R^{+}} $: 在工作 $j \in J$ 位置开始服务的最晚时间。

$dd_{j} \in \mathbb{R^{+}} $: 工作 $j \in J$ 的截止日期,即完成服务的最晚时间。

$M \in \mathbb{N}$: 这是一个非常大的数字。该数字确定如下:计划期间为10个工作小时(即600分钟)。我们希望 $M$ 的值比计划期间大一个数量级。因此,我们设定M = 6100。

### 决策变量

$x_{j,k} \in \{0,1\}$: 如果工作 $j \in J$ 分配给技术人员 $k \in K$,则该变量为1;否则为0。

$u_{k} \in \{0,1\}$: 如果技术人员 $k \in K$ 被用于执行工作,则该变量为1;否则为0。

$y_{i,j,k} \in \{0,1\}$: 如果技术人员 $k \in K$ 从位置 $i \in L$ 到位置 $j \in L$ 旅行,则该变量为1;否则为0。

$t_{j} \geq 0$: 该变量确定在位置 $j \in J$ 到达或开始服务的时间。

$z_{j} \geq 0$: 该变量确定完成工作 $j \in J$ 的延迟时间。

$xa_{j}, xb_{j} \geq 0$: 修正工作 $j \in J$ 的最早和最晚开始服务时间。

$g_{j}$: 如果工作 $j \in J$ 无法完成,则该变量为1;否则为0。

### 目标函数

- **最小化延迟:** 目标函数是最小化所有工作的总加权延迟。

\begin{equation}
\sum_{j \in J} \pi_{j} \cdot z_{j} + \sum_{j \in J} 0.01 \cdot \pi_{j} \cdot M(xa_{j} + xb_{j}) + 
\sum_{j \in J} \pi_{j} \cdot M \cdot g_{j}
\tag{0}
\end{equation}

**注意**: 我们希望将填补工作的需求约束和在时间窗口内开始工作的约束视为软约束。软约束可以被违反,但违反将导致巨大的惩罚。为了考虑每个工作有不同的优先级,我们使用优先级惩罚来计算与违反软约束相关的惩罚。我们假设未能满足工作的需求会极大地降低客户满意度,因此应承担与软约束相关的最大惩罚。回想一下参数 $M$ 的值是一个大数字,那么与未能完成工作相关的惩罚如下: $\pi_{j} \cdot M$。时间窗口约束的违反也会降低客户满意度,但程度低于与时间窗口约束相关的惩罚。时间窗口约束如下确定: $0.01 \cdot \pi_{j} \cdot M$。

### 约束条件

- **分配合格的技术人员:** 对于每个工作,我们分配一个有资格的技术人员,或者声明一个缺口。

\begin{equation}
\sum_{k \in K(j)} x_{j,k} + g_{j} = 1 \quad \forall j \in J
\tag{1}
\end{equation}

**注意**: 缺口变量 $g_{j}$ 的惩罚是 ($0.1 \cdot \pi_{j} \cdot M$),这是一个大数字,以阻止
无法满足需求。

- **只有一个技术人员:** 对于每个工作,我们只允许分配一个技术人员。

\begin{equation}
\sum_{k \in K} x_{j,k} \leq 1 \quad \forall j \in J
\tag{2}
\end{equation}

- **技术人员容量:** 对于每个技术人员,我们确保技术人员的可用容量不被超过。

\begin{equation}
\sum_{j \in J}  p_{j} \cdot x_{j,k} + \sum_{i \in L} \sum_{j \in L} \tau_{i,j} \cdot y_{i,j,k} \leq W_{k} \cdot u_{k} \quad \forall k \in K
\tag{3}
\end{equation}

- **技术人员行程:** 对于每个技术人员和工作,我们确保如果技术人员被分配到工作,则技术人员必须前往另一个位置(形成行程)。

\begin{equation}
\sum_{j \in L}  y_{i,j,k} = x_{i,k} \quad \forall i \in J, k \in K
\tag{4}
\end{equation}

- 对于每个技术人员和工作,我们确保如果技术人员被分配到工作,则技术人员必须从另一个位置前往工作的地点(形成行程)。

\begin{equation}
\sum_{i \in L}  y_{i,j,k} = x_{j,k} \quad \forall j \in J, k \in K
\tag{5}
\end{equation}

- **相同仓库:** 对于每个技术人员和仓库,我们确保如果技术人员被分配到任何工作,则必须从其所在的服务中心(仓库)出发并返回。

\begin{equation}
\sum_{j \in J}  y_{d,j,k} = \beta_{k,d} \cdot u_{k} \quad \forall  k \in K, d \in D
\tag{6}
\end{equation}

\begin{equation}
\sum_{i \in J}  y_{i,d,k} = \beta_{k,d} \cdot u_{k} \quad \forall  k \in K, d \in D
\tag{7}
\end{equation}

- **时间关系:** 对于每个位置和工作,我们确保由同一技术人员服务的两个连续工作的时间关系。即,如果技术人员 $k$ 从工作 $i$ 旅行到工作 $j$,则在工作 $j$ 开始服务的时间必须不小于工作 $i$ 的完成时间加上从工作 $i$ 到工作 $j$ 的旅行时间。

\begin{equation}
t_{j} \geq t_{i} + p_{i} + \tau_{i,j} - M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) \quad \forall i \in L, j \in J
\tag{8}
\end{equation}

**注意**: 请注意,如果技术人员 $k$ 从工作 $i$ 的位置旅行到工作 $j$ 的位置,则
$\sum_{k \in K}  y_{i,j,k} = 1$。因此, $M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) = 0$, 约束
$t_{j} \geq t_{i} + p_{i} + \tau_{i,j}$ 将被正确执行。现在考虑技术人员 $k$ 不从工作 $i$ 的位置旅行到工作 $j$ 的位置的情况。因此, $\sum_{k \in K}  y_{i,j,k} = 0$ 和
$M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) = M$。在这种情况下,该约束变为
$t_{j} \geq t_{i} + p_{i} + \tau_{i,j} - M$。但 $M$ 是一个非常大的数字,因此 $t_{i} + p_{i} + \tau_{i,j} - M < 0$ 和
由于 $t_{j} \geq 0$, 该约束是多余的。

- **时间窗口:** 对于每个工作 $j \in J$ 确保工作的时间窗口得到满足。

\begin{equation}
t_{j} \geq a_{j} - xa_{j} \quad \forall j \in J
\tag{9}
\end{equation}

\begin{equation}
t_{j} \leq b_{j} + xb_{j} \quad \forall j \in J
\tag{10}
\end{equation}

**注意**: 为了阻止违反工作的时间窗口,我们将惩罚
($0.01 \cdot \pi_{j} \cdot M$) 与修正变量 $xa_{j}, xb_{j}$ 相关联。

- **延迟约束:** 对于每个工作 $j \in J$ 计算工作的延迟时间。

\begin{equation}
z_{j} \geq t_{j} + p_{j} - dd_{j} \quad \forall j \in J
\tag{11}
\end{equation}

- 请注意,由于延迟决策变量 $z_{j}$ 是非负的,在截止日期前完成工作没有好处;另一方面,由于目标函数最小化总加权延迟,约束(11)应始终是约束性的。

## 问题实例

在这个场景中,我们考虑为下一工作日安排和调度技术人员的问题,以最小化客户预约的延迟。电信公司有七名技术人员:Albert, Bob, Carlos, Doris, Ed, Flor 和 Gina。有两个服务中心:海德堡和弗赖堡。技术人员仅在这些服务中心之一工作。每个技术人员的可用小时数和服务中心基站(仓库)如下表所示。

| <i></i> | Albert | Bob | Carlos | Doris | Ed | Flor | Gina |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 分钟 | 480 | 480 | 480 | 480 | 480 | 360 | 360 |
| 仓库 | 海德堡 | 海德堡 | 弗赖堡 | 弗赖堡 | 海德堡 | 弗赖堡 | 海德堡 |

电信公司有不同类型的工作。
下表显示了工作类型的优先级(4为最高优先级,1为最低优先级)和持续时间(分钟)。

| <i></i> | 优先级 | 持续时间(分钟) |
| --- | --- | --- | 
| 设备安装 | 2 | 60 |
| 设备设置 | 3 | 30 |
| 检查/服务设备 | 1 | 60 |
| 常规维修 | 1 | 60 |
| 重要维修 | 2 | 120 |
| 紧急维修 | 3 | 90 |
| 关键维修 | 4 | 60 |

下表显示了每个技术人员有资格执行的工作。

| <i></i> | Albert | Bob | Carlos | Doris | Ed | Flor | Gina |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 设备安装 | - | - | - | 1 | - | 1 | 1 |
| 设备设置 | 1 | 1 | 1 | 1 | - | - | 1 |
| 检查/服务设备 | 1 | - | 1 | - | 1 | - | - |
| 常规维修 | - | 1 | 1 | - | 1 | 1 | 1 |
| 重要维修 | - | - | - | 1 | - | 1 | 1 |
| 紧急维修 | - | 1 | 1 | - | 1 | 1 | 1 |
| 关键维修 | - | - | - | 1 | - | - | 1 |

电信公司接收客户对特定工作类型、预约(截止)时间和技术人员可以到达的服务时间窗口的请求。下表显示了客户的订单及其要求。对于每个客户,指定了位置。

| <i></i> | C1:曼海姆  | C2:卡尔斯鲁厄  | C3:巴登-巴登  | C4:比尔  | C5:奥芬堡 | C6:拉尔/黑森林 | C7:洛拉赫 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 工作类型 | 设备设置 | 设备设置 | 常规维修 | 设备安装 | 设备安装 | 关键维修 | 检查/服务设备 |
| 截止时间 | 8:00 | 10:00 | 11:00  | 12:00 | 14:00 | 15:00 | 16:00 |
| 时间窗口 | 7:00-7:30 | 7:30-9:30 | 8:00-10:00 | 9:00-11:00 | 11:00-13:00 | 12:00-14:00 | 13:00-15:00 |

计划期间为7:00到17:00,即10小时。时间段以分钟为单位,然后截止时间和时间窗口将转换为从0分钟开始到600分钟结束。例如,对于客户C2,截止时间为10:00(180分钟),时间窗口为7:30到9:30(30分钟到150分钟)。

下表显示了从任何仓库或客户位置到任何仓库或客户位置的旅行时间(分钟)。

| <i></i> | 海德堡 | 弗赖堡 | 曼海姆  | 卡尔斯鲁厄  | 巴登-巴登  | 比尔  | 奥芬堡 | 拉尔/黑森林 | 洛拉赫 |
| --- | --- | --- | --- | --- | --- | --- | --- |--- |--- |
| 海德堡 | - | 120 | 24 | 50 | 67 | 71 | 88 | 98 | 150 |
| 弗赖堡 | - | - | 125 | 85 | 68 | 62 | 45 | 39 | 48 |
| 曼海姆 | - | - | - | 53 | 74 | 77 | 95 | 106 | 160 |
| 卡尔斯鲁厄 | - | - | - | - | 31 | 35 | 51 | 61 | 115 |
| 巴登-巴登 | - | - | - | - | - | 16 | 36 | 46 | 98 |
| 比尔 | - | - | - | - | - | - | 30 | 40 | 92 |
| 奥芬堡 | - | - | - | - | - | - | - | 26 | 80 |
| 拉尔/黑森林 | - | - | - | - | - | - | - | - | 70 |
| 洛拉赫 | - | - | - | - | - | - | - | - | - |


## Python 实现

在这个Jupyter Notebook中,我们使用以下库:

* `sys` 访问系统特定的参数和函数。
* `pandas` 从Excel文件读取数据。
* `gurobipy` Gurobi优化器库。

此实现基于面向对象编程。

### 辅助类

以下类适当地组织了MIP模型的输入数据。

* `Technician`。
* `Job`。
* `Customer`。

### 辅助函数

* `solve_trs0` 构建并解决MIP模型。
* `printScen` 打印输出报告的标题。


In [ ]:
# %pip install gurobipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v10.0.2

In [2]:
class Technician():
    def __init__(self, name, cap, depot):
        self.name = name
        self.cap = cap
        self.depot = depot

    def __str__(self):
        return f"Technician: {self.name}\n  Capacity: {self.cap}\n  Depot: {self.depot}"

In [3]:
class Job():
    def __init__(self, name, priority, duration, coveredBy):
        self.name = name
        self.priority = priority
        self.duration = duration
        self.coveredBy = coveredBy

    def __str__(self):
        about = f"Job: {self.name}\n  Priority: {self.priority}\n  Duration: {self.duration}\n  Covered by: "
        about += ", ".join([t.name for t in self.coveredBy])
        return about

In [4]:
class Customer():
    def __init__(self, name, loc, job, tStart, tEnd, tDue):
        self.name = name
        self.loc = loc
        self.job = job
        self.tStart = tStart
        self.tEnd = tEnd
        self.tDue = tDue

    def __str__(self):
        coveredBy = ", ".join([t.name for t in self.job.coveredBy])
        return f"Customer: {self.name}\n  Location: {self.loc}\n  Job: {self.job.name}\n  Priority: {self.job.priority}\n  Duration: {self.job.duration}\n  Covered by: {coveredBy}\n  Start time: {self.tStart}\n  End time: {self.tEnd}\n  Due time: {self.tDue}"


## 基本场景

对于基本场景,有足够的技术人员容量来满足所有客户的需求。要运行基本场景,请考虑电子表格文件 `data-Sce0.xlsx` 并在打开Excel工作簿时插入此名称。

In [5]:
# 读取Excel工作簿
excel_file = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/technician_routing_scheduling/data-Sce0.xlsx'
df = pd.read_excel(excel_file, sheet_name='Technicians')
df = df.rename(columns={df.columns[0]: "name", df.columns[1]: "cap", df.columns[2]: "depot"})

df1 = df.drop(df.columns[3:], axis=1).drop(df.index[[0,1]])
# 创建Technician对象
technicians = [Technician(*row) for row in df1.itertuples(index=False, name=None)]
# print(df1)

In [7]:
# 读取工作数据
jobs=[]
for j in range(3, len(df.columns)):
    coveredBy = [t for i, t in enumerate(technicians) if df.iloc[2+i,j]==1]
    thisJob = Job(df.iloc[2:,j].name, df.iloc[0,j], df.iloc[1,j], coveredBy)
    jobs.append(thisJob)

In [8]:
# 读取位置数据
df_locations = pd.read_excel(excel_file, sheet_name='Locations', index_col=0) #, skiprows=1, index_col=0)

# 提取位置并初始化距离字典
locations = df_locations.index
dist = {(l, l): 0 for l in locations}

# 填充距离字典
for i, l1 in enumerate(locations):
    for j, l2 in enumerate(locations):
        if i < j:
            dist[l1, l2] = df_locations.iloc[i, j]
            dist[l2, l1] = dist[l1, l2]

In [10]:
# 读取客户数据
df_customers = pd.read_excel(excel_file, sheet_name='Customers')

customers = []
for i, c in enumerate(df_customers.iloc[:, 0]):
    job_name = df_customers.iloc[i, 2]
    
    # 查找相应的Job对象
    matching_job = next((job for job in jobs if job.name == job_name), None)
    
    if matching_job is not None:
        # 使用相应的Job对象创建Customer对象
        this_customer = Customer(c, df_customers.iloc[i, 1], matching_job, *df_customers.iloc[i, 3:])
        customers.append(this_customer)

In [ ]:
def solve_trs0(technicians, customers, dist):
    # 构建有用的数据结构
    K = [k.name for k in technicians]
    C = [j.name for j in customers]
    J = [j.loc for j in customers]
    L = list(set([l[0] for l in dist.keys()]))
    D = list(set([t.depot for t in technicians]))
    cap = {k.name : k.cap for k in technicians}
    loc = {j.name : j.loc for j in customers}
    depot = {k.name : k.depot for k in technicians}
    canCover = {j.name : [k.name for k in j.job.coveredBy] for j in customers}
    dur = {j.name : j.job.duration for j in customers}
    tStart = {j.name : j.tStart for j in customers}
    tEnd = {j.name : j.tEnd for j in customers}
    tDue = {j.name : j.tDue for j in customers}
    priority = {j.name : j.job.priority for j in customers}
    
        ### 创建模型
    m = gp.Model("trs0")
    
    ### 决策变量
    # 客户-技术人员分配
    x = m.addVars(C, K, vtype=GRB.BINARY, name="x")
    
    # 技术人员分配
    u = m.addVars(K, vtype=GRB.BINARY, name="u")
    
    # 边路由分配给技术人员
    y = m.addVars(L, L, K, vtype=GRB.BINARY, name="y")
   
    # 技术人员不能离开或返回不是其基站的仓库
    for k in technicians:
        for d in D:
            if k.depot != d:
                for i in L:
                    y[i,d,k.name].ub = 0
                    y[d,i,k.name].ub = 0
    
    # 服务开始时间
    t = m.addVars(L, ub=600, name="t")
    
    # 服务延迟
    z = m.addVars(C, name="z")
    
    # 修正时间窗口上下限的人工变量
    xa = m.addVars(C, name="xa")
    xb = m.addVars(C, name="xb")
    
    # 未完成的工作
    g = m.addVars(C, vtype=GRB.BINARY, name="g")
    
        ### 约束条件

    # 必须分配一个技术人员到工作,或者声明一个缺口(1)
    m.addConstrs((gp.quicksum(x[j, k] for k in canCover[j]) + g[j] == 1 for j in C), name="assignToJob")
    
    # 最多一个技术人员可以分配到工作(2)
    m.addConstrs((x.sum(j, '*') <= 1 for j in C), name="assignOne")

    # 技术人员容量约束(3)
    capLHS = {k : gp.quicksum(dur[j]*x[j,k] for j in C) +\
        gp.quicksum(dist[i,j]*y[i,j,k] for i in L for j in L) for k in K}
    m.addConstrs((capLHS[k] <= cap[k]*u[k] for k in K), name="techCapacity")

    # 技术人员行程约束(4和5)
    m.addConstrs((y.sum('*', loc[j], k) == x[j,k] for k in K for j in C),\
        name="techTour1")
    m.addConstrs((y.sum(loc[j], '*', k) == x[j,k] for k in K for j in C),\
        name="techTour2")

    # 相同仓库约束(6和7)
    m.addConstrs((gp.quicksum(y[j,depot[k],k] for j in J) == u[k] for k in K),\
        name="sameDepot1")
    m.addConstrs((gp.quicksum(y[depot[k],j,k] for j in J) == u[k] for k in K),\
        name="sameDepot2")

    # 客户位置的时间约束(8)
    M = {(i,j) : 600 + dur[i] + dist[loc[i], loc[j]] for i in C for j in C}
    m.addConstrs((t[loc[j]] >= t[loc[i]] + dur[i] + dist[loc[i], loc[j]]\
        - M[i,j]*(1 - gp.quicksum(y[loc[i],loc[j],k] for k in K))\
        for i in C for j in C), name="tempoCustomer")

    # 仓库位置的时间约束(8)
    M = {(i,j) : 600 + dist[i, loc[j]] for i in D for j in C}
    m.addConstrs((t[loc[j]] >= t[i] + dist[i, loc[j]]\
        - M[i,j]*(1 - y.sum(i,loc[j],'*')) for i in D for j in C),\
        name="tempoDepot")

    # 时间窗口约束(9和10)
    m.addConstrs((t[loc[j]] + xa[j] >= tStart[j] for j in C), name="timeWinA")
    m.addConstrs((t[loc[j]] - xb[j] <= tEnd[j] for j in C), name="timeWinB")

    # 延迟约束(11)
    m.addConstrs((z[j] >= t[loc[j]] + dur[j] - tDue[j] for j in C),\
        name="lateness")

    ### 目标函数
    M = 6100
    
    m.setObjective(z.prod(priority) + gp.quicksum( 0.01 * M * priority[j] * (xa[j] + xb[j]) for j in C) +
                   gp.quicksum( M * priority[j] * g[j] for j in C) , GRB.MINIMIZE)
    
    # m.write("TRS0.lp")
    m.optimize()

    status = m.Status
    if status in [GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED]:
        print("模型不可行或无界。")
        sys.exit(0)
    elif status != GRB.OPTIMAL:
        print("优化终止,状态为 {}".format(status))
        sys.exit(0)
        
    ### 打印结果
    # 分配
    print("")
    for j in customers:
        if g[j.name].X > 0.5:
            jobStr = "没有人分配给 {} ({}) 在 {}".format(j.name,j.job.name,j.loc)
        else:
            for k in K:
                if x[j.name,k].X > 0.5:
                    jobStr = "{} 分配给 {} ({}) 在 {}。开始时间 t={:.2f}。".format(k,j.name,j.job.name,j.loc,t[j.loc].X)
                    if z[j.name].X > 1e-6:
                        jobStr += " 延迟 {:.2f} 分钟。".format(z[j.name].X)
                    if xa[j.name].X > 1e-6:
                        jobStr += " 开始时间修正 {:.2f} 分钟。".format(xa[j.name].X)
                    if xb[j.name].X > 1e-6:
                        jobStr += " 结束时间修正 {:.2f} 分钟。".format(xb[j.name].X)
        print(jobStr)

    # 技术人员
    print("")
    for k in technicians:
        if u[k.name].X > 0.5:
            cur = k.depot
            route = k.depot
            while True:
                for j in customers:
                    if y[cur,j.loc,k.name].X > 0.5:
                        route += " -> {} (距离={}, t={:.2f}, 处理={})".format(j.loc, dist[cur,j.loc], t[j.loc].X, j.job.duration)
                        cur = j.loc
                for i in D:
                    if y[cur,i,k.name].X > 0.5:
                        route += " -> {} (距离={})".format(i, dist[cur,i])
                        cur = i
                        break
                if cur == k.depot:
                    break
            print("{} 的路线: {}".format(k.name, route))
        else:
            print("{} 未使用".format(k.name)) 
            
    
    # 利用率
    print("")
    for k in K:
        used = capLHS[k].getValue()
        total = cap[k]
        util = used / cap[k] if cap[k] > 0 else 0
        print("{} 的利用率为 {:.2%} ({:.2f}/{:.2f})".format(k, util,\
            used, cap[k]))
    totUsed = sum(capLHS[k].getValue() for k in K)
    totCap = sum(cap[k] for k in K)
    totUtil = totUsed / totCap if totCap > 0 else 0
    print("总技术人员利用率为 {:.2%} ({:.2f}/{:.2f})".format(totUtil, totUsed, totCap))
    
    m.dispose()
    gp.disposeDefaultEnv()
    

In [12]:
def printScen(scenStr):
    sLen = len(scenStr)
    print("\n" + "*"*sLen + "\n" + scenStr + "\n" + "*"*sLen + "\n")


In [13]:
if __name__ == "__main__":
    # 基本模型
    printScen("解决基本场景模型")
    solve_trs0(technicians, customers, dist)



********
解决基本场景模型
********

Set parameter LicenseID to value 2601452
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 217 rows, 660 columns and 2329 nonzeros
Model fingerprint: 0x458241dd
Variable types: 30 continuous, 630 integer (630 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [1e+00, 2e+04]
  Bounds range     [1e+00, 6e+02]
  RHS range        [1e+00, 6e+02]
Found heuristic solution: objective 245830.00000
Presolve removed 28 rows and 189 columns
Presolve time: 0.01s
Presolved: 189 rows, 471 columns, 2090 nonzeros
Variable types: 26 continuous, 445 integer (445 binary)

Root relaxation: objective 0.000000e+00, 79 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl U

## 基本场景分析
对于基本场景,我们有足够的技术人员容量来满足客户需求。请注意,最优目标函数值为0.00,这意味着所有工作都已完成,它们按时完成,并且没有对时间窗口限制进行修正。

第一个报告描述了分配给每个客户的技术人员、客户的位置和工作的开始时间。第二个报告描述了每个分配的技术人员的路线。路线定义了*起始位置*和
*目标位置*。参数*距离*定义了从*起始位置*到*目标位置*所需的分钟数。参数t确定了在*目标位置*开始工作的*开始时间*。参数*处理*显示了技术人员完成工作的分钟数。
第三个报告描述了每个技术人员的容量利用率和所有技术人员的总体容量利用率。技术人员的容量利用率是技术人员花费在驾驶或服务客户上的分钟数除以技术人员的容量。所有技术人员的总体容量利用率是所有技术人员花费在驾驶或服务客户上的总分钟数除以所有可用技术人员的总容量。

## 场景1

我们假设技术人员的工作容量减半。此外,我们假设客户5需要紧急服务,其截止时间为00分钟。由于截止时间已更改,我们删除时间窗口限制,即时间窗口在计划期间的任何时间。新的时间窗口为[0, 600]。

要运行此场景,请考虑电子表格文件 `data-Sce3.xlsx` 并在打开Excel工作簿时插入此名称。


In [14]:
# 打开Excel工作簿
excel_file = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/technician_routing_scheduling/data-Sce3.xlsx'

# 读取技术人员数据
df = pd.read_excel(excel_file, sheet_name='Technicians')
df = df.rename(columns={df.columns[0]: "name", df.columns[1]: "cap", df.columns[2]: "depot"})

df1 = df.drop(df.columns[3:], axis=1).drop(df.index[[0,1]])
# 创建Technician对象
technicians = [Technician(*row) for row in df1.itertuples(index=False, name=None)]

In [15]:
# 读取工作数据
jobs=[]
for j in range(3, len(df.columns)):
    coveredBy = [t for i, t in enumerate(technicians) if df.iloc[2+i,j]==1]
    thisJob = Job(df.iloc[2:,j].name, df.iloc[0,j], df.iloc[1,j], coveredBy)
    jobs.append(thisJob)

In [16]:
# 读取位置数据
df_locations = pd.read_excel(excel_file, sheet_name='Locations', index_col=0) #, skiprows=1, index_col=0)

# 提取位置并初始化距离字典
locations = df_locations.index
dist = {(l, l): 0 for l in locations}

# 填充距离字典
for i, l1 in enumerate(locations):
    for j, l2 in enumerate(locations):
        if i < j:
            dist[l1, l2] = df_locations.iloc[i, j]
            dist[l2, l1] = dist[l1, l2]

In [17]:
# 读取客户数据
df_customers = pd.read_excel(excel_file, sheet_name='Customers')

customers = []
for i, c in enumerate(df_customers.iloc[:, 0]):
    job_name = df_customers.iloc[i, 2]
    
    # 查找相应的Job对象
    matching_job = next((job for job in jobs if job.name == job_name), None)
    
    if matching_job is not None:
        # 使用相应的Job对象创建Customer对象
        this_customer = Customer(c, df_customers.iloc[i, 1], matching_job, *df_customers.iloc[i, 3:])
        customers.append(this_customer)

In [18]:
if __name__ == "__main__":
    # 基本模型
    printScen("解决场景1模型")
    solve_trs0(technicians, customers, dist)


*******
解决场景1模型
*******

Set parameter LicenseID to value 2601452
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 217 rows, 660 columns and 2329 nonzeros
Model fingerprint: 0xaf1ba1d6
Variable types: 30 continuous, 630 integer (630 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [1e+00, 2e+04]
  Bounds range     [1e+00, 6e+02]
  RHS range        [1e+00, 6e+02]
Found heuristic solution: objective 216670.00000
Presolve removed 168 rows and 593 columns
Presolve time: 0.01s
Presolved: 49 rows, 67 columns, 279 nonzeros
Found heuristic solution: objective 55020.000000
Variable types: 15 continuous, 52 integer (52 binary)

Root relaxation: objective 8.214242e+03, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |   

## 场景1分析

我们假设技术人员的工作容量减半。此外,我们假设客户5需要紧急服务,其截止时间为00分钟。由于截止时间已更改,我们删除时间窗口限制,即时间窗口在计划期间的任何时间。新的时间窗口为[0, 600]。

* 七个工作中有六个被填补。
  * 客户5的服务将延迟105分钟。
  * 开始时间为45分钟,在时间窗口[0, 600]内。
* 客户4的服务需求无法填补。
  * 请注意,该客户的优先级较低。
* 六名技术人员被分配以满足6个客户的需求。
* Ed未分配给任何客户。
  * 请注意,Ed不具备客户4(设备安装)的工作资格。
* 总体技术人员利用率为54.23%。

## 结论

在这个Jupyter Notebook中,我们使用Gurobi Python API公式化并解决了具有时间窗口约束的多仓库车辆路径问题。这个MIP公式是电信行业的一个优化应用;然而,该公式非常通用,可以很容易地适应运输和物流行业的任何类型的车辆路径问题。

我们解决的路由和调度问题考虑了一家运营多个服务中心为其客户提供服务的电信公司。我们通过同时做出三种决策来解决这个路由和调度问题:
- (i) 在所有服务中心分配工作给技术人员
- (ii) 每个技术人员的路由,即技术人员访问客户的顺序
- (iii) 工作的调度,即技术人员到达客户位置并完成相应工作的时间。

电信公司的目标是最小化所有工作的总加权延迟。

在这个笔记本中,我们讨论了两个场景,基本场景有足够的技术人员容量来按时满足所有客户的需求。在场景1中,我们将所有技术人员的容量减少了一半,并且一个客户有紧急情况需要在早上首先服务。该场景捕捉了MIP模型在技术人员容量有限和客户工作优先级和要求的情况下需要考虑的各种权衡。

## 参考文献
[1] S. Salhi, A. Imran, N. A. Wassan. *The multi-depot vehicle routing problem with heterogeneous vehicle fleet: Formulation and a variable neighborhood search implementation*. Computers & Operations Research 52 (2014) 315-325.

版权所有 © 2020 Gurobi Optimization, LLC